# Sonar landmine detection system 

### This models uses data from Sonar satellite. The data is composed of only rock and mine categories. The purpose of this model is to classify whether an object is a rock or a landmine 

#### Import required data analysis libraries 

In [14]:
import numpy as np
import pandas as pd
from math import sqrt

#### Import classification models 

In [15]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier


#### Import model_selection metrics 

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score


#### Set global variables 

In [17]:
pd.set_option('display.max_columns', None)

#### Load the dataset 

In [18]:
sonar_data = pd.read_csv(r'E:\dev\ML\IBM\Data science\Data\SONAR - Rock vs Mine prediction model\sonarData.csv', header=None)
sonar_data.head()

0       1       2       3       4       5       6       7       8   \
0  0.0286  0.0453  0.0277  0.0174  0.0384  0.0990  0.1201  0.1833  0.2105   
1  0.0317  0.0956  0.1321  0.1408  0.1674  0.1710  0.0731  0.1401  0.2083   
2  0.0519  0.0548  0.0842  0.0319  0.1158  0.0922  0.1027  0.0613  0.1465   
3  0.0223  0.0375  0.0484  0.0475  0.0647  0.0591  0.0753  0.0098  0.0684   
4  0.0164  0.0173  0.0347  0.0070  0.0187  0.0671  0.1056  0.0697  0.0962   

       9       10      11      12      13      14      15      16      17  \
0  0.3039  0.2988  0.4250  0.6343  0.8198  1.0000  0.9988  0.9508  0.9025   
1  0.3513  0.1786  0.0658  0.0513  0.3752  0.5419  0.5440  0.5150  0.4262   
2  0.2838  0.2802  0.3086  0.2657  0.3801  0.5626  0.4376  0.2617  0.1199   
3  0.1487  0.1156  0.1654  0.3833  0.3598  0.1713  0.1136  0.0349  0.3796   
4  0.0251  0.0801  0.1056  0.1266  0.0890  0.0198  0.1133  0.2826  0.3234   

       18      19      20      21      22      23      24      25      26  \
0  0.7234  0.5122  0.2074  0.3985  0.5890  0.2872  0.2043  0.5782  0.5389   
1  0.2024  0.4233  0.7723  0.9735  0.9390  0.5559  0.5268  0.6826  0.5713   
2  0.6676  0.9402  0.7832  0.5352  0.6809  0.9174  0.7613  0.8220  0.8872   
3  0.7401  0.9925  0.9802  0.8890  0.6712  0.4286  0.3374  0.7366  0.9611   
4  0.3238  0.4333  0.6068  0.7652  0.9203  0.9719  0.9207  0.7545  0.8289   

       27      28      29      30      31      32      33      34      35  \
0  0.3750  0.3411  0.5067  0.5580  0.4778  0.3299  0.2198  0.1407  0.2856   
1  0.5429  0.2177  0.2149  0.5811  0.6323  0.2965  0.1873  0.2969  0.5163   
2  0.6091  0.2967  0.1103  0.1318  0.0624  0.0990  0.4006  0.3666  0.1050   
3  0.7353  0.4856  0.1594  0.3007  0.4096  0.3170  0.3305  0.3408  0.2186   
4  0.8907  0.7309  0.6896  0.5829  0.4935  0.3101  0.0306  0.0244  0.1108   

       36      37      38      39      40      41      42      43      44  \
0  0.3807  0.4158  0.4054  0.3296  0.2707  0.2650  0.0723  0.1238  0.1192   
1  0.6153  0.4283  0.5479  0.6133  0.5017  0.2377  0.1957  0.1749  0.1304   
2  0.1915  0.3930  0.4288  0.2546  0.1151  0.2196  0.1879  0.1437  0.2146   
3  0.2463  0.2726  0.1680  0.2792  0.2558  0.1740  0.2121  0.1099  0.0985   
4  0.1594  0.1371  0.0696  0.0452  0.0620  0.1421  0.1597  0.1384  0.0372   

       45      46      47      48      49      50      51      52      53  \
0  0.1089  0.0623  0.0494  0.0264  0.0081  0.0104  0.0045  0.0014  0.0038   
1  0.0597  0.1124  0.1047  0.0507  0.0159  0.0195  0.0201  0.0248  0.0131   
2  0.2360  0.1125  0.0254  0.0285  0.0178  0.0052  0.0081  0.0120  0.0045   
3  0.1271  0.1459  0.1164  0.0777  0.0439  0.0061  0.0145  0.0128  0.0145   
4  0.0688  0.0867  0.0513  0.0092  0.0198  0.0118  0.0090  0.0223  0.0179   

       54      55      56      57      58      59 60  
0  0.0013  0.0089  0.0057  0.0027  0.0051  0.0062  R  
1  0.0070  0.0138  0.0092  0.0143  0.0036  0.0103  R  
2  0.0121  0.0097  0.0085  0.0047  0.0048  0.0053  R  
3  0.0058  0.0049  0.0065  0.0093  0.0059  0.0022  R  
4  0.0084  0.0068  0.0032  0.0035  0.0056  0.0040  R

#### Get the number of datapoints in the dataset and also its statistics 

In [19]:
print(sonar_data.shape)
sonar_data.describe()

(184, 61)


0           1           2           3           4           5   \
count  184.000000  184.000000  184.000000  184.000000  184.000000  184.000000   
mean     0.029550    0.039225    0.044495    0.055378    0.076540    0.104729   
std      0.023744    0.034382    0.040040    0.048552    0.057853    0.059652   
min      0.001500    0.000600    0.001500    0.005800    0.006700    0.010200   
25%      0.013350    0.016050    0.018625    0.024375    0.036800    0.067025   
50%      0.022650    0.030400    0.034300    0.044250    0.062500    0.091350   
75%      0.036350    0.049550    0.060025    0.070025    0.107050    0.136050   
max      0.137100    0.233900    0.305900    0.426400    0.401000    0.382300   

               6           7           8           9           10          11  \
count  184.000000  184.000000  184.000000  184.000000  184.000000  184.000000   
mean     0.120978    0.132159    0.174284    0.202479    0.228570    0.240743   
std      0.063577    0.086000    0.119492    0.136812    0.133916    0.139723   
min      0.003300    0.005500    0.007500    0.011300    0.028900    0.023600   
25%      0.077100    0.074025    0.093850    0.101275    0.119725    0.120000   
50%      0.105500    0.110100    0.145500    0.173900    0.214100    0.236850   
75%      0.155150    0.170450    0.231475    0.270150    0.297750    0.326775   
max      0.372900    0.459000    0.682800    0.710600    0.734200    0.577100   

               12          13          14          15          16          17  \
count  184.000000  184.000000  184.000000  184.000000  184.000000  184.000000   
mean     0.271335    0.304139    0.333299    0.394330    0.432828    0.465770   
std      0.142513    0.163731    0.206587    0.233147    0.265620    0.268489   
min      0.025200    0.042000    0.003100    0.016200    0.034900    0.037500   
25%      0.158575    0.178675    0.169625    0.203500    0.208200    0.240375   
50%      0.251650    0.286500    0.299650    0.345500    0.350150    0.381400   
75%      0.365325    0.394275    0.473550    0.541825    0.680125    0.689625   
max      0.713100    0.997000    1.000000    0.998800    1.000000    1.000000   

               18          19          20          21          22          23  \
count  184.000000  184.000000  184.000000  184.000000  184.000000  184.000000   
mean     0.516160    0.573876    0.617445    0.627629    0.644652    0.663184   
std      0.263078    0.267487    0.264196    0.265125    0.259632    0.244392   
min      0.049400    0.065600    0.051200    0.021900    0.056300    0.023900   
25%      0.309375    0.362050    0.406775    0.407575    0.436850    0.528075   
50%      0.443300    0.555950    0.651750    0.668900    0.689650    0.694350   
75%      0.758475    0.821000    0.838325    0.849850    0.865425    0.864200   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

               24          25          26          27          28          29  \
count  184.000000  184.000000  184.000000  184.000000  184.000000  184.000000   
mean     0.660883    0.685720    0.680198    0.667598    0.622658    0.574539   
std      0.245745    0.233565    0.244669    0.228440    0.241313    0.227638   
min      0.024000    0.092100    0.048100    0.028400    0.014400    0.061300   
25%      0.510225    0.538150    0.497725    0.510875    0.453900    0.399000   
50%      0.701650    0.732700    0.702900    0.691900    0.629600    0.576650   
75%      0.856300    0.878900    0.884625    0.857700    0.835775    0.748250   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

               30          31          32          33          34          35  \
count  184.000000  184.000000  184.000000  184.000000  184.000000  184.000000   
mean     0.497792    0.438137    0.427920    0.424439    0.417841    0.413477   
std      0.218812    0.220258    0.212586    0.233199    0.258855    0.259679   
min      0.048200    0.040400    0.047700

#### Get the number of rocks and mines 

In [20]:
sonar_data[60].value_counts()

R    92
M    92
Name: 60, dtype: int64

#### We have 111 mines and 97 rocks. We need to classify this 60th column as a category 

In [21]:
sonar_data[60] = sonar_data[60].astype('category')
sonar_data.dtypes

0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
56     float64
57     float64
58     float64
59     float64
60    category
Length: 61, dtype: object

#### Convert the M & R labels into zeros and ones  using cat.codes

In [22]:
sonar_data[60] = sonar_data[60].cat.codes
sonar_data

0       1       2       3       4       5       6       7       8   \
0    0.0286  0.0453  0.0277  0.0174  0.0384  0.0990  0.1201  0.1833  0.2105   
1    0.0317  0.0956  0.1321  0.1408  0.1674  0.1710  0.0731  0.1401  0.2083   
2    0.0519  0.0548  0.0842  0.0319  0.1158  0.0922  0.1027  0.0613  0.1465   
3    0.0223  0.0375  0.0484  0.0475  0.0647  0.0591  0.0753  0.0098  0.0684   
4    0.0164  0.0173  0.0347  0.0070  0.0187  0.0671  0.1056  0.0697  0.0962   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
179  0.0269  0.0383  0.0505  0.0707  0.1313  0.2103  0.2263  0.2524  0.3595   
180  0.0340  0.0625  0.0381  0.0257  0.0441  0.1027  0.1287  0.1850  0.2647   
181  0.0209  0.0191  0.0411  0.0321  0.0698  0.1579  0.1438  0.1402  0.3048   
182  0.0368  0.0279  0.0103  0.0566  0.0759  0.0679  0.0970  0.1473  0.2164   
183  0.0089  0.0274  0.0248  0.0237  0.0224  0.0845  0.1488  0.1224  0.1569   

         9       10      11      12      13      14      15      16      17  \
0    0.3039  0.2988  0.4250  0.6343  0.8198  1.0000  0.9988  0.9508  0.9025   
1    0.3513  0.1786  0.0658  0.0513  0.3752  0.5419  0.5440  0.5150  0.4262   
2    0.2838  0.2802  0.3086  0.2657  0.3801  0.5626  0.4376  0.2617  0.1199   
3    0.1487  0.1156  0.1654  0.3833  0.3598  0.1713  0.1136  0.0349  0.3796   
4    0.0251  0.0801  0.1056  0.1266  0.0890  0.0198  0.1133  0.2826  0.3234   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
179  0.5915  0.6675  0.5679  0.5175  0.3334  0.2002  0.2856  0.2937  0.3424   
180  0.4117  0.5245  0.5341  0.5554  0.3915  0.2950  0.3075  0.3021  0.2719   
181  0.3914  0.3504  0.3669  0.3943  0.3311  0.3331  0.3002  0.2324  0.1381   
182  0.2544  0.2936  0.2935  0.2657  0.3187  0.2794  0.2534  0.1980  0.1929   
183  0.2119  0.3003  0.3094  0.2743  0.2547  0.1870  0.1452  0.1457  0.2429   

         18      19      20      21      22      23      24      25      26  \
0    0.7234  0.5122  0.2074  0.3985  0.5890  0.2872  0.2043  0.5782  0.5389   
1    0.2024  0.4233  0.7723  0.9735  0.9390  0.5559  0.5268  0.6826  0.5713   
2    0.6676  0.9402  0.7832  0.5352  0.6809  0.9174  0.7613  0.8220  0.8872   
3    0.7401  0.9925  0.9802  0.8890  0.6712  0.4286  0.3374  0.7366  0.9611   
4    0.3238  0.4333  0.6068  0.7652  0.9203  0.9719  0.9207  0.7545  0.8289   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
179  0.5949  0.7526  0.8959  0.8147  0.7109  0.7378  0.7201  0.8254  0.8917   
180  0.5443  0.7932  0.8751  0.8667  0.7107  0.6911  0.7287  0.8792  1.0000   
181  0.3450  0.4428  0.4890  0.3677  0.4379  0.4864  0.6207  0.7256  0.6624   
182  0.2826  0.3245  0.3504  0.3324  0.4217  0.4774  0.4808  0.6325  0.8334   
183  0.3259  0.3679  0.3355  0.3100  0.3914  0.5280  0.6409  0.7707  0.8754   

         27      28      29      30      31      32      33      34      35  \
0    0.3750  0.3411  0.5067  0.5580  0.4778  0.3299  0.2198  0.1407  0.2856   
1    0.5429  0.2177  0.2149  0.5811  0.6323  0.2965  0.1873  0.2969  0.5163   
2    0.6091  0.2967  0.1103  0.1318  0.0624  0.0990  0.4006  0.3666  0.1050   
3    0.7353  0.4856  0.1594  0.3007  0.4096  0.3170  0.3305  0.3408  0.2186   
4    0.8907  0.7309  0.6896  0.5829  0.4935  0.3101  0.0306  0.0244  0.1108   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
179  0.9820  0.8179  0.4848  0.3203  0.2775  0.2382  0.2911  0.1675  0.3156   
180  0.9816  0.8984  0.6048  0.4934  0.5371  0.4586  0.2908  0.0774  0.2249   
181  0.7689  0.7981  0.8577  0.9273  0.7009  0.4851  0.3409  0.1406  0.1147   
182  0.9458  1.0000  0.8425  0.5524  0.4795  0.5200  0.3968  0.1940  0.1519   
183  1.0000  0.9806  0.6969  0.4973  0.5020  0.5359  0.3842  0.1848  0.1149   

         36      37      38      39      40      41      42      43      44  \
0    0.3807  0.4158  0.4054  0.3296  0.2707  0.2650  0.0723  0.1238  0.1192   
1    0.6153  0.4283  0.5479  0.6133  0.5017  0.2377  0.

   ### 1 is a Rock.      
   ### 0 is a Mine

#### Lets now split the data for machine learning algorithms 

In [23]:
# We are selecting the whole dataset but without the 60th column which  we are dropping with an axis of 1
X = sonar_data.drop(columns = 60, axis = 1)
Y = sonar_data[60]

print(X.shape, Y.shape)

(184, 60) (184,)


#### Assign the data points into train and test chunks using the train_test_split 

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, stratify=Y, random_state=10)

print(X_train.shape,X_test.shape, Y_train.shape, Y_test.shape)

(147, 60) (37, 60) (147,) (37,)


In [25]:
models = [
    LogisticRegression(),
    RandomForestClassifier(),
    SVC(kernel='linear'),
    DecisionTreeClassifier(),
    KNeighborsClassifier(),
    GradientBoostingClassifier()
]

In [26]:
def run_models_with_train_test_split():
    for model in models:
        model.fit(X_train, Y_train)
        training_prediction = model.predict(X_train)
        training_prediction_accuracy = accuracy_score(training_prediction , Y_train)
        testing_prediction = model.predict(X_test)
        testing_prediction_accuracy = accuracy_score(testing_prediction, Y_test)
        print('Accuracy on the training data for : ', model , 'is ----', training_prediction_accuracy)
        print('Accuracy on the testing data for : ' , model , 'is ----', testing_prediction_accuracy)
        print('*************************************************************************************')

In [27]:
run_models_with_train_test_split()

Accuracy on the training data for :  LogisticRegression() is ---- 0.8639455782312925
Accuracy on the testing data for :  LogisticRegression() is ---- 0.6216216216216216
*************************************************************************************
Accuracy on the training data for :  RandomForestClassifier() is ---- 1.0
Accuracy on the testing data for :  RandomForestClassifier() is ---- 0.6486486486486487
*************************************************************************************
Accuracy on the training data for :  SVC(kernel='linear') is ---- 0.8775510204081632
Accuracy on the testing data for :  SVC(kernel='linear') is ---- 0.7027027027027027
*************************************************************************************
Accuracy on the training data for :  DecisionTreeClassifier() is ---- 1.0
Accuracy on the testing data for :  DecisionTreeClassifier() is ---- 0.6756756756756757
********************************************************************************

In [32]:
def check_errors_with_train_test_split():
    for model in models:
        model.fit(X_train, Y_train)
        training_prediction = model.predict(X_train)
        testing_prediction = model.predict(X_test)
        print('Root Mean Squared Error on TRAINING data for ----', model ,' is ---', sqrt(mean_squared_error(Y_train, training_prediction)))
        print('Root Mean Squared Error on TESTING data for ----' , model , ' is ---', sqrt(mean_squared_error(Y_test, testing_prediction)))
        print('Mean Absolute Error on TRAINING data for ---- ', model , ' is ---', mean_absolute_error(Y_train,training_prediction))
        print('Mean Absolute Error on TESTING data for ----', model , ' is ---', mean_absolute_error(Y_test, testing_prediction))
        print('*************************************************************************************')

In [33]:
check_errors_with_train_test_split()

Root Mean Squared Error on TRAINING data for ---- LogisticRegression()  is --- 0.36885555678165877
Root Mean Squared Error on TESTING data for ---- LogisticRegression()  is --- 0.6151246852292456
Mean Absolute Error on TRAINING data for ----  LogisticRegression()  is --- 0.1360544217687075
Mean Absolute Error on TESTING data for ---- LogisticRegression()  is --- 0.3783783783783784
*************************************************************************************
Root Mean Squared Error on TRAINING data for ---- RandomForestClassifier()  is --- 0.0
Root Mean Squared Error on TESTING data for ---- RandomForestClassifier()  is --- 0.5927489783638191
Mean Absolute Error on TRAINING data for ----  RandomForestClassifier()  is --- 0.0
Mean Absolute Error on TESTING data for ---- RandomForestClassifier()  is --- 0.35135135135135137
*************************************************************************************
Root Mean Squared Error on TRAINING data for ---- SVC(kernel='linear')  is

### Run models with cross-validation 

In [34]:
def run_models_with_kfold_cross_validation():
    for model in models:
        cv_score = cross_val_score(model, X, Y, cv=10)
        mean_accuracy = sum(cv_score) / len(cv_score)
        mean_accuracy = mean_accuracy * 100
        mean_accuracy = round(mean_accuracy, 2)
        print (' Accuracy for ---', model , 'is ---', cv_score)
        print ('Mean accuracy for ---', model , 'is ---', mean_accuracy ,'%')
        print ('*************************************************************')

In [35]:
run_models_with_kfold_cross_validation()

 Accuracy for --- LogisticRegression() is --- [0.47368421 0.52631579 0.73684211 1.         0.55555556 0.88888889
 0.66666667 0.66666667 0.38888889 0.94444444]
Mean accuracy for --- LogisticRegression() is --- 68.48 %
*************************************************************
 Accuracy for --- RandomForestClassifier() is --- [0.68421053 0.42105263 0.78947368 0.94736842 0.72222222 0.83333333
 0.83333333 0.5        0.44444444 0.94444444]
Mean accuracy for --- RandomForestClassifier() is --- 71.2 %
*************************************************************
 Accuracy for --- SVC(kernel='linear') is --- [0.47368421 0.52631579 0.68421053 1.         0.61111111 0.88888889
 0.61111111 0.72222222 0.5        0.94444444]
Mean accuracy for --- SVC(kernel='linear') is --- 69.62 %
*************************************************************
 Accuracy for --- DecisionTreeClassifier() is --- [0.73684211 0.52631579 0.73684211 0.73684211 0.44444444 0.77777778
 0.66666667 0.72222222 0.5        0.722

### RandomForestClassifier is performing better than other classifiers. Note that this is an ensemble algorithm powered by multiple decision trees  

#### We select RandomForestClassifier as our production model 

In [36]:
production_model = SVC(kernel='linear')

In [37]:
production_model.fit(X_train, Y_train)

SVC(kernel='linear')

## Testing the production model with unseen data 

In [38]:
unseen_testing_data = [
    [0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,0.3108,0.2933,0.2275,0.0994,0.1801,0.2200,0.2732,0.2862,0.2034,0.1740,0.4130,0.6879,0.8120,0.8453,0.8919,0.9300,0.9987,1.0000,0.8104,0.6199,0.6041,0.5547,0.4160,0.1472,0.0849,0.0608,0.0969,0.1411,0.1676,0.1200,0.1201,0.1036,0.1977,0.1339,0.0902,0.1085,0.1521,0.1363,0.0858,0.0290,0.0203,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157],
    [0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032],
    [0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,0.3085,0.3425,0.2990,0.1402,0.1235,0.1534,0.1901,0.2429,0.2120,0.2395,0.3272,0.5949,0.8302,0.9045,0.9888,0.9912,0.9448,1.0000,0.9092,0.7412,0.7691,0.7117,0.5304,0.2131,0.0928,0.1297,0.1159,0.1226,0.1768,0.0345,0.1562,0.0824,0.1149,0.1694,0.0954,0.0080,0.0790,0.1255,0.0647,0.0179,0.0051,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067],
    [0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044],
    [0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,0.2716,0.2374,0.1878,0.0983,0.0683,0.1503,0.1723,0.2339,0.1962,0.1395,0.3164,0.5888,0.7631,0.8473,0.9424,0.9986,0.9699,1.0000,0.8630,0.6979,0.7717,0.7305,0.5197,0.1786,0.1098,0.1446,0.1066,0.1440,0.1929,0.0325,0.1490,0.0328,0.0537,0.1309,0.0910,0.0757,0.1059,0.1005,0.0535,0.0235,0.0155,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031],
    [0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078],
    [0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,0.2898,0.2812,0.1578,0.0273,0.0673,0.1444,0.2070,0.2645,0.2828,0.4293,0.5685,0.6990,0.7246,0.7622,0.9242,1.0000,0.9979,0.8297,0.7032,0.7141,0.6893,0.4961,0.2584,0.0969,0.0776,0.0364,0.1572,0.1823,0.1349,0.0849,0.0492,0.1367,0.1552,0.1548,0.1319,0.0985,0.1258,0.0954,0.0489,0.0241,0.0042,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048],
    [0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117],
    [0.0260,0.0363,0.0136,0.0272,0.0214,0.0338,0.0655,0.1400,0.1843,0.2354,0.2720,0.2442,0.1665,0.0336,0.1302,0.1708,0.2177,0.3175,0.3714,0.4552,0.5700,0.7397,0.8062,0.8837,0.9432,1.0000,0.9375,0.7603,0.7123,0.8358,0.7622,0.4567,0.1715,0.1549,0.1641,0.1869,0.2655,0.1713,0.0959,0.0768,0.0847,0.2076,0.2505,0.1862,0.1439,0.1470,0.0991,0.0041,0.0154,0.0116,0.0181,0.0146,0.0129,0.0047,0.0039,0.0061,0.0040,0.0036,0.0061,0.0115],
    [0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094],
    [0.0272,0.0378,0.0488,0.0848,0.1127,0.1103,0.1349,0.2337,0.3113,0.3997,0.3941,0.3309,0.2926,0.1760,0.1739,0.2043,0.2088,0.2678,0.2434,0.1839,0.2802,0.6172,0.8015,0.8313,0.8440,0.8494,0.9168,1.0000,0.7896,0.5371,0.6472,0.6505,0.4959,0.2175,0.0990,0.0434,0.1708,0.1979,0.1880,0.1108,0.1702,0.0585,0.0638,0.1391,0.0638,0.0581,0.0641,0.1044,0.0732,0.0275,0.0146,0.0091,0.0045,0.0043,0.0043,0.0098,0.0054,0.0051,0.0065,0.0103],
    [0.0335,0.0258,0.0398,0.0570,0.0529,0.1091,0.1709,0.1684,0.1865,0.2660,0.3188,0.3553,0.3116,0.1965,0.1780,0.2794,0.2870,0.3969,0.5599,0.6936,0.7969,0.7452,0.8203,0.9261,0.8810,0.8814,0.9301,0.9955,0.8576,0.6069,0.3934,0.2464,0.1645,0.1140,0.0956,0.0080,0.0702,0.0936,0.0894,0.1127,0.0873,0.1020,0.1964,0.2256,0.1814,0.2012,0.1688,0.1037,0.0501,0.0136,0.0130,0.0120,0.0039,0.0053,0.0062,0.0046,0.0045,0.0022,0.0005,0.0031],
    [0.0131,0.0387,0.0329,0.0078,0.0721,0.1341,0.1626,0.1902,0.2610,0.3193,0.3468,0.3738,0.3055,0.1926,0.1385,0.2122,0.2758,0.4576,0.6487,0.7154,0.8010,0.7924,0.8793,1.0000,0.9865,0.9474,0.9474,0.9315,0.8326,0.6213,0.3772,0.2822,0.2042,0.2190,0.2223,0.1327,0.0521,0.0618,0.1416,0.1460,0.0846,0.1055,0.1639,0.1916,0.2085,0.2335,0.1964,0.1300,0.0633,0.0183,0.0137,0.0150,0.0076,0.0032,0.0037,0.0071,0.0040,0.0009,0.0015,0.0085],
    [0.0116,0.0744,0.0367,0.0225,0.0076,0.0545,0.1110,0.1069,0.1708,0.2271,0.3171,0.2882,0.2657,0.2307,0.1889,0.1791,0.2298,0.3715,0.6223,0.7260,0.7934,0.8045,0.8067,0.9173,0.9327,0.9562,1.0000,0.9818,0.8684,0.6381,0.3997,0.3242,0.2835,0.2413,0.2321,0.1260,0.0693,0.0701,0.1439,0.1475,0.0438,0.0469,0.1476,0.1742,0.1555,0.1651,0.1181,0.0720,0.0321,0.0056,0.0202,0.0141,0.0103,0.0100,0.0034,0.0026,0.0037,0.0044,0.0057,0.0035],
    [0.0238,0.0318,0.0422,0.0399,0.0788,0.0766,0.0881,0.1143,0.1594,0.2048,0.2652,0.3100,0.2381,0.1918,0.1430,0.1735,0.1781,0.2852,0.5036,0.6166,0.7616,0.8125,0.7793,0.8788,0.8813,0.9470,1.0000,0.9739,0.8446,0.6151,0.4302,0.3165,0.2869,0.2017,0.1206,0.0271,0.0580,0.1262,0.1072,0.1082,0.0360,0.1197,0.2061,0.2054,0.1878,0.2047,0.1716,0.1069,0.0477,0.0170,0.0186,0.0096,0.0071,0.0084,0.0038,0.0026,0.0028,0.0013,0.0035,0.0060],
    [0.0366,0.0421,0.0504,0.0250,0.0596,0.0252,0.0958,0.0991,0.1419,0.1847,0.2222,0.2648,0.2508,0.2291,0.1555,0.1863,0.2387,0.3345,0.5233,0.6684,0.7766,0.7928,0.7940,0.9129,0.9498,0.9835,1.0000,0.9471,0.8237,0.6252,0.4181,0.3209,0.2658,0.2196,0.1588,0.0561,0.0948,0.1700,0.1215,0.1282,0.0386,0.1329,0.2331,0.2468,0.1960,0.1985,0.1570,0.0921,0.0549,0.0194,0.0166,0.0132,0.0027,0.0022,0.0059,0.0016,0.0025,0.0017,0.0027,0.0027],
    [0.0050,0.0017,0.0270,0.0450,0.0958,0.0830,0.0879,0.1220,0.1977,0.2282,0.2521,0.3484,0.3309,0.2614,0.1782,0.2055,0.2298,0.3545,0.6218,0.7265,0.8346,0.8268,0.8366,0.9408,0.9510,0.9801,0.9974,1.0000,0.9036,0.6409,0.3857,0.2908,0.2040,0.1653,0.1769,0.1140,0.0740,0.0941,0.0621,0.0426,0.0572,0.1068,0.1909,0.2229,0.2203,0.2265,0.1766,0.1097,0.0558,0.0142,0.0281,0.0165,0.0056,0.0010,0.0027,0.0062,0.0024,0.0063,0.0017,0.0028],
    [0.0129,0.0141,0.0309,0.0375,0.0767,0.0787,0.0662,0.1108,0.1777,0.2245,0.2431,0.3134,0.3206,0.2917,0.2249,0.2347,0.2143,0.2939,0.4898,0.6127,0.7531,0.7718,0.7432,0.8673,0.9308,0.9836,1.0000,0.9595,0.8722,0.6862,0.4901,0.3280,0.3115,0.1969,0.1019,0.0317,0.0756,0.0907,0.1066,0.1380,0.0665,0.1475,0.2470,0.2788,0.2709,0.2283,0.1818,0.1185,0.0546,0.0219,0.0204,0.0124,0.0093,0.0072,0.0019,0.0027,0.0054,0.0017,0.0024,0.0029],
    [0.0392,0.0108,0.0267,0.0257,0.0410,0.0491,0.1053,0.1690,0.2105,0.2471,0.2680,0.3049,0.2863,0.2294,0.1165,0.2127,0.2062,0.2222,0.3241,0.4330,0.5071,0.5944,0.7078,0.7641,0.8878,0.9711,0.9880,0.9812,0.9464,0.8542,0.6457,0.3397,0.3828,0.3204,0.1331,0.0440,0.1234,0.2030,0.1652,0.1043,0.1066,0.2110,0.2417,0.1631,0.0769,0.0723,0.0912,0.0812,0.0496,0.0101,0.0089,0.0083,0.0080,0.0026,0.0079,0.0042,0.0071,0.0044,0.0022,0.0014],
    [0.0203,0.0121,0.0380,0.0128,0.0537,0.0874,0.1021,0.0852,0.1136,0.1747,0.2198,0.2721,0.2105,0.1727,0.2040,0.1786,0.1318,0.2260,0.2358,0.3107,0.3906,0.3631,0.4809,0.6531,0.7812,0.8395,0.9180,0.9769,0.8937,0.7022,0.6500,0.5069,0.3903,0.3009,0.1565,0.0985,0.2200,0.2243,0.2736,0.2152,0.2438,0.3154,0.2112,0.0991,0.0594,0.1940,0.1937,0.1082,0.0336,0.0177,0.0209,0.0134,0.0094,0.0047,0.0045,0.0042,0.0028,0.0036,0.0013,0.0016],
    [0.0056,0.0267,0.0221,0.0561,0.0936,0.1146,0.0706,0.0996,0.1673,0.1859,0.2481,0.2712,0.2934,0.2637,0.1880,0.1405,0.2028,0.2613,0.2778,0.3346,0.3830,0.4003,0.5114,0.6860,0.7490,0.7843,0.9021,1.0000,0.8888,0.6511,0.6083,0.4463,0.2948,0.1729,0.1488,0.0801,0.1770,0.1382,0.2404,0.2046,0.1970,0.2778,0.1377,0.0685,0.0664,0.1665,0.1807,0.1245,0.0516,0.0044,0.0185,0.0072,0.0055,0.0074,0.0068,0.0084,0.0037,0.0024,0.0034,0.0007],
    [0.0315,0.0252,0.0167,0.0479,0.0902,0.1057,0.1024,0.1209,0.1241,0.1533,0.2128,0.2536,0.2686,0.2803,0.1886,0.1485,0.2160,0.2417,0.2989,0.3341,0.3786,0.3956,0.5232,0.6913,0.7868,0.8337,0.9199,1.0000,0.8990,0.6456,0.5967,0.4355,0.2997,0.2294,0.1866,0.0922,0.1829,0.1743,0.2452,0.2407,0.2518,0.3184,0.1685,0.0675,0.1186,0.1833,0.1878,0.1114,0.0310,0.0143,0.0138,0.0108,0.0062,0.0044,0.0072,0.0007,0.0054,0.0035,0.0001,0.0055]
    
]

In [39]:
def test_production_model_with_new_data():
    for new_data in unseen_testing_data:
        new_input_data_as_numpy_array = np.asarray(new_data)
        reshaped_new_input_data = new_input_data_as_numpy_array.reshape(1, -1)
        prediction = production_model.predict(reshaped_new_input_data)
        print(prediction)
        if prediction[0] == 0:
            print('THIS IS A LANDMINE')
            
        else:
            print('THIS IS A ROCK')
            
        print('************************************8')    

In [40]:
test_production_model_with_new_data()

[0]
THIS IS A LANDMINE
************************************8
[0]
THIS IS A LANDMINE
************************************8
[1]
THIS IS A ROCK
************************************8
[0]
THIS IS A LANDMINE
************************************8
[1]
THIS IS A ROCK
************************************8
[0]
THIS IS A LANDMINE
************************************8
[1]
THIS IS A ROCK
************************************8
[0]
THIS IS A LANDMINE
************************************8
[1]
THIS IS A ROCK
************************************8
[0]
THIS IS A LANDMINE
************************************8
[0]
THIS IS A LANDMINE
************************************8
[0]
THIS IS A LANDMINE
************************************8
[0]
THIS IS A LANDMINE
************************************8
[0]
THIS IS A LANDMINE
************************************8
[0]
THIS IS A LANDMINE
************************************8
[0]
THIS IS A LANDMINE
************************************8
[0]
THIS IS A LANDMINE
*****************